In [178]:
# Load library
import fairsearchcore as fsc
import pandas as pd
dataframen = pd.read_csv("ONE_samlet_preds_index_ny_WO.csv")
dataframen

,Unnamed: 0,Idx,Gender_bins,Preds
0,0,12016,[0],93.777222
1,1,11864,[1],93.685806
2,2,8893,[0],93.681221
3,3,4900,[0],93.660019
4,4,12018,[1],93.660019
...,...,...,...,...
4091,4091,9028,[1],18.230223
4092,4092,1043,[0],17.869230
4093,4093,4014,[1],17.439417
4094,4094,8687,[0],15.629829


## NY FA*IR funktion: Maximize utility while maximizing fairness. 




In [179]:
df_try = dataframen#[:10000]
df_try

,Unnamed: 0,Idx,Gender_bins,Preds
0,0,12016,[0],93.777222
1,1,11864,[1],93.685806
2,2,8893,[0],93.681221
3,3,4900,[0],93.660019
4,4,12018,[1],93.660019
...,...,...,...,...
4091,4091,9028,[1],18.230223
4092,4092,1043,[0],17.869230
4093,4093,4014,[1],17.439417
4094,4094,8687,[0],15.629829


In [180]:
# If WO run this
# df_try['Gender_bins'] = df_try['Gender_bins'].str.strip('[]').astype(int)
# df_try

In [181]:
import warnings

In [182]:
def our_FAIR(df, prop, n_prefix, k):
    
    """" 
    Our fair algorithm inspired by Zehlike's FA*IR:
    df is predicted df. prop is desired proportion. n_prefix is desired prop for each n prefix, 
    k is top k ranking from df of length N. 
    
    Output is 'result' of a fair top-k ranking with desired proportion for the whole df and in every prefix
    
    """
    


    #we subtract by 0, because we are using 0-index
    n_prefix = int(n_prefix *k)-1

    # if k/n_prefix != int(k/n_prefix):
    #     warnings.warn('Please use 0.1 as 10 percent of the top-k as prefix')
    
    #split df in two df groups: protected and non-protected and ensure each of them are sorted
    #in descending order by Preds scores
    
    #protected (1=Female)
    dfp = df[df.Gender_bins == 1].sort_values(by='Preds', ascending=False).reset_index()
    #non-protected
    dfpn = df[df.Gender_bins == 0].sort_values(by='Preds', ascending=False).reset_index()
    
    #dfnew with of the top-k fair sorting of org. df
    #new df
    dfnew = pd.DataFrame(columns = df.columns)
    
    #print OBS if it's not possible to achieve the given prop for each n prefix for the whole top-k
    
    #for-loop for creating ranking in batches of prefix_n through k
    
    #desired number of protected candidates in each n batch
    prop_ok = int(n_prefix*prop)
    prop_pn = int(n_prefix - prop_ok)
    
    #index in each dataframe
    dfp_Idx = 0
    dfpn_Idx = 0
    
    for i in range(int(k/n_prefix)): #loop over number of n_prefixes in length k
        p_Idx = 0
        pn_Idx = 0

        for j in range(n_prefix):
            if dfp.Preds.loc[dfp_Idx] > dfpn.Preds.loc[dfpn_Idx]:
                dfnew = dfnew.append(dfp.loc[dfp_Idx])
                p_Idx += 1
                dfp_Idx += 1
            else:
                dfnew = dfnew.append(dfpn.loc[dfpn_Idx])
                pn_Idx += 1
                dfpn_Idx += 1

            #if the proportion of nonprotected group is achieved within the prefix, add rest as protected 
            if pn_Idx == prop_pn:
                dfnew = dfnew.append(dfp.loc[dfp_Idx : dfp_Idx + (prop_ok-p_Idx)])
                dfp_Idx += (prop_ok-p_Idx)
                break
            #if the proportion of protected group is achieved within the prefix, add rest as non-protected
            elif p_Idx == prop_ok:
                dfnew = dfnew.append(dfpn.loc[dfpn_Idx : dfpn_Idx + (prop_pn-pn_Idx) ])
                dfpn_Idx += (prop_pn-pn_Idx)
                break

        dfnew = dfnew.drop(["index"], axis=1).reset_index()
                
    return dfnew



In [183]:
ress = our_FAIR(df_try, 0.40, 0.1, 500)
# sum(ress.Gender_bins[630:700])
# ress

KeyError: 0

Check ratio of women out of total

In [ ]:
# sum(ress.Gender_bins)/len(ress)
print(ress.Gender_bins.value_counts())
print(len(ress[ress.Gender_bins == 1]))
print(len(ress))
# print(len(ress[ress.Gender_bins == 1])/len(ress))
print('{0:.05f}'.format(len(ress[ress.Gender_bins == 1])/len(ress)))

0.0    300
1.0    200
Name: Gender_bins, dtype: int64
200
500
0.40000


## Save this as a reranking and plot the entropy and utility

In [ ]:
re_ranked_WO = pd.DataFrame(ress, columns = ['Idx','Gender_bins','Preds'])
re_ranked_WO.to_csv("re_ranked_WO.csv")

# re_ranked_PLAIN = pd.DataFrame(ress, columns = ['Idx','Gender_bins','Preds'])
# re_ranked_PLAIN.to_csv("re_ranked_PLAIN.csv")